In [5]:
import torch
mps = torch.device("mps")

In [6]:
values = torch.tensor([1,2,3,4,5,6], dtype=torch.float32, device=mps)
starts = torch.tensor([0,3,5], dtype=torch.int32,device=mps)

In [8]:
bc = torch.cumsum(values, dim=0)

In [9]:
print(bc)

tensor([ 1.,  3.,  6., 10., 15., 21.], device='mps:0')


In [15]:
baselines = torch.zeros_like(starts, dtype=torch.float32)
baselines[1:] = bc[starts[1:]-1]
print(baselines)

tensor([ 0.,  6., 15.], device='mps:0')


In [25]:
seg_ids = torch.bucketize(torch.arange(len(values), dtype=torch.int32, device=mps), starts, right=True)-1

In [26]:
baselines[seg_ids]

tensor([ 0.,  0.,  0.,  6.,  6., 15.], device='mps:0')

In [27]:
bc - baselines[seg_ids]

tensor([1., 3., 6., 4., 9., 6.], device='mps:0')

In [28]:
# This gives us the segment scan. But what we really want are just the per-segment sums

In [35]:
bc[[2,4,5]] - torch.tensor([0, 6,15],device=mps)

tensor([6., 9., 6.], device='mps:0')

In [37]:
ends = torch.cat([starts[1:], torch.tensor([len(values)], device=starts.device)])
print(ends)

tensor([3, 5, 6], device='mps:0')


In [38]:
bc[ends-1]

tensor([ 6., 15., 21.], device='mps:0')

In [39]:
bc[[2,4,5]]

tensor([ 6., 15., 21.], device='mps:0')

In [43]:
bc[ends-1] - torch.cat([torch.tensor([0.], device=mps), bc[starts[1:]-1]])

tensor([6., 9., 6.], device='mps:0')

In [44]:
# Full code for segment_sum, given cumsum offsets

values = torch.tensor([1,2,3,4,5,6], dtype=torch.float32, device=mps)
starts = torch.tensor([0,3,5], dtype=torch.int32,device=mps)

bc = torch.cumsum(values, dim=0)
ends = torch.cat([starts[1:], torch.tensor([len(values)], device=starts.device)])
segsum = bc[ends-1] - torch.cat([torch.tensor([0.], device=mps), bc[starts[1:]-1]])

print(segsum)

tensor([6., 9., 6.], device='mps:0')
